# Player Characteristics and Subscribing to a Game Related Newsletter

## Introduction

!!!

First, we will import the necessary libraries for our analysis.

In [1]:
import altair as alt
import pandas as pd
import numpy as np
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

alt.data_transformers.enable('vegafusion')

set_config(transform_output="pandas")

In [2]:
players = pd.read_csv("https://drive.google.com/uc?export=download&id=1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz")
players

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN
...,...,...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,NaN,NaN
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,NaN,NaN
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,NaN,NaN


## Method and Results

Before starting to create the model, we will explore this dataset to find the best variables to use for our model. We are interested in whether certain player types can effect subscription status. From the dataset loaded above, players can be described by their age, number of hours they played the game, and their experience level. This is a good starting point at picking predictor variables for our model. But first, visualizing the relationship between these variables and if a player is subcribed or not will clarify if there is any meaningful relationship between the predictors and our target. Figure 1 compares the numerical predictors of hours played and age while also showing each player's experience level and subscription status.

In [16]:
players_plot_1 = alt.Chart(players).mark_point(opacity = 0.6).encode(
    x = alt.X("age").title("Age"),
    y = alt.Y("played_hours").title("Time Played (Hours)").scale(type = "sqrt"),
    shape = alt.Shape("subscribe").title("Subscribed"),
    color = alt.Color("experience").title("Experience")
).properties(
    title = "Figure 1: Age vs Time Played for Various Player Types",
    width = 700,
    height = 500
)

players_plot_1

alt.Chart(...)

From this plot, it is clear that there is no trend between player experience level and whether they are subscribed or not as given by each points' colour and shape respectively. Hence, for our model we will not be looking at player experience level as an indicator to our target variable, subcription status. To assess whether time played and age have any effects on subscription, we can simplify this graph and reduce noise by omitting the experience level colouring and instead colouring by whether a player has subscribed.

In [13]:
players_plot_2 = alt.Chart(players).mark_point(opacity = 0.6).encode(
    x = alt.X("age").title("Age"),
    y = alt.Y("played_hours").title("Time Played (Hours)").scale(type = "sqrt"),
    color = alt.Color("subscribe").title("Subscribed")
).properties(
    title = "Figure 2: Age vs Time Played of All Players",
    width = 700,
    height = 500
)

players_plot_2

alt.Chart(...)

Looking at this new plot, we can see a few trends. Many points marked as a circle (indicating that a player is subscribed) tend to fall past the age of 25 or above 15 hours played. Therefore, it is safe to assume that both player age and the amount of time they played contribute to if they choose to subcribe to a game related newsletter. We will be using both of these variables as predictor variables in our classification model to predictor our target variable, 'subcribe'.

Moving ahead, we will not need the other columns of our dataset, hence we will drop them. Other than this slight modification, we will not need to wrangle our data any further since it is already tidy. This will be the dataset we use for our model.

In [4]:
players_reduced = players.drop(columns = ["experience", "individualId", "organizationName", "hashedEmail", "name", "gender"])
players_reduced

,subscribe,played_hours,age
0,True,30.3,9
1,True,3.8,17
2,False,0.0,17
3,True,0.7,21
4,True,0.1,21
...,...,...,...
191,True,0.0,17
192,False,0.3,22
193,False,0.0,17
194,False,2.3,17


To start the model, we create a preprocessor to standardize each of 'played_hours' and 'age' since they are on varying scales. Then we combine this preprocessor in a pipeline with a classifier model of no specific number of neighbors so that we can grid search for the best 'n' value ourselves.

In [5]:
np.random.seed(2025)

players_train, players_test = train_test_split(players_reduced, train_size = 0.75, stratify = players_reduced["subscribe"])

X_train = players_train[["played_hours", "age"]]
y_train = players_train["subscribe"]

X_test = players_test[["played_hours", "age"]]
y_test = players_test["subscribe"]

players_preprocessor = make_column_transformer(
    (StandardScaler(), ["played_hours", "age"]),
    remainder = "passthrough",
)

knn = KNeighborsClassifier()

players_pipe = make_pipeline(players_preprocessor, knn)

players_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['played_hours', 'age'])])),
                ('kneighborsclassifier', KNeighborsClassifier())])

## Discussion

!!!

 To do: use regressor, scale, test/split, choose k, find RMSPE, predict, plot

To do: discussion

References???